<a href="https://colab.research.google.com/github/Aleixbs/Python_bases/blob/main/PlaywrightScrapingCatastro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install playwright

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 21.6 MB/s eta 0:00:00


In [7]:
!playwright install

167.7 MiB [] 0% 127.6s167.7 MiB [] 0% 279.1s167.7 MiB [] 0% 284.4s167.7 MiB [] 0% 186.4s167.7 MiB [] 0% 171.1s167.7 MiB [] 0% 128.2s167.7 MiB [] 0% 103.4s167.7 MiB [] 0% 79.1s167.7 MiB [] 0% 62.4s167.7 MiB [] 0% 64.2s167.7 MiB [] 0% 49.8s167.7 MiB [] 0% 39.5s167.7 MiB [] 0% 37.6s167.7 MiB [] 1% 29.1s167.7 MiB [] 1% 25.2s167.7 MiB [] 1% 21.9s167.7 MiB [] 2% 17.1s167.7 MiB [] 2% 14.3s167.7 MiB [] 3% 12.3s167.7 MiB [] 4% 9.8s167.7 MiB [] 5% 8.6s167.7 MiB [] 5% 8.0s167.7 MiB [] 6% 7.7s167.7 MiB [] 7% 6.7s167.7 MiB [] 7% 6.2s167.7 MiB [] 8% 5.9s167.7 MiB [] 9% 5.5s167.7 MiB [] 9% 5.3s167.7 MiB [] 10% 5.0s167.7 MiB [] 11% 4.8s167.7 MiB [] 11% 4.7s167.7 MiB [] 12% 4.5s167.7 MiB [] 13% 4.3s167.7 MiB [] 14% 4.1s167.7 MiB [] 15% 4.0s167.7 MiB [] 15% 3.8s167.7 MiB [] 16% 3.7s167.7 MiB [] 17% 3.6s167.7 MiB [] 18% 3.5s167.7 MiB [] 19% 3.4s167.7 MiB [] 19% 3.3s167.7 MiB [] 20% 3.2s167.7 MiB [] 21% 3.2s167.7 MiB [] 21% 3.1s167.7 MiB [] 22% 3.0s167.7 MiB [] 23% 3.0s167.7 MiB [] 24% 2.9s167.7 MiB [] 24

In [8]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Go to Spanish cadastre site
        await page.goto("https://www1.sedecatastro.gob.es/Cartografia/Visor.aspx")

        # Wait for page to load fully
        await page.wait_for_timeout(5000)

        print("✅ Page loaded successfully")

        await browser.close()

# This should run properly now
await main()


✅ Page loaded successfully


In [10]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    refcat = "9923301YJ2792D0001FP"
    base_url = "https://www1.sedecatastro.gob.es/OVCFrames.aspx?TIPO=CONSULTA"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()

        # Go to search page
        await page.goto(base_url)

        # Wait for the iframe "main" to be attached
        await page.wait_for_selector('iframe[name="main"]')

        # Get the frame reference
        frame = page.frame(name="main")

        # Click "Consulta por referencia catastral"
        await frame.locator('text=Referencia Catastral').click()

        # Fill in the refcat field (the input is split in 2 parts: first 14 digits + 5 digits)
        await frame.fill('input[name="RefC"]', refcat[:14])
        await frame.fill('input[name="RCBice2"]', refcat[14:])

        # Submit form
        await frame.locator('input[type="submit"]').click()

        # Wait for search results to load
        await frame.wait_for_selector('text=Datos del inmueble')

        # Try to locate facade photo link
        photo_link = await frame.locator('text=Foto de fachada').get_attribute('href')
        if photo_link:
            photo_url = "https://www1.sedecatastro.gob.es" + photo_link
            print("✅ Image URL found:", photo_url)
        else:
            print("❌ No image available for this parcel.")

        await browser.close()

await main()



TimeoutError: Page.wait_for_selector: Timeout 30000ms exceeded.
Call log:
  - waiting for locator("iframe[name=\"main\"]") to be visible


In [13]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    refcat = "9923301YJ2792D0001FP"
    base_url = "https://www1.sedecatastro.gob.es/OVCConsultaRC/OVCConsultaRC.aspx"

    async with async_playwright() as p:
        # browser = await p.chromium.launch(headless=False)  # headless False helps a LOT
        browser = await p.chromium.launch(headless=True) # only for collab

        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            locale="es-ES"
        )
        page = await context.new_page()

        await page.goto(base_url, timeout=60000)
        await page.wait_for_selector('input[name="tbRC1"]', timeout=30000)

        await page.fill('input[name="tbRC1"]', refcat[:14])
        await page.fill('input[name="tbRC2"]', refcat[14:])
        await page.click('input[name="btBuscar"]')

        await page.wait_for_selector('text=Datos del Inmueble', timeout=30000)

        photo_link = await page.locator('a:has-text("Foto de Fachada")').get_attribute('href')
        if photo_link:
            photo_url = "https://www1.sedecatastro.gob.es" + photo_link
            print("✅ Image URL found:", photo_url)
        else:
            print("❌ No image available for this parcel.")

        await browser.close()

await main()


TargetClosedError: BrowserType.launch: Target page, context or browser has been closed
Browser logs:

╔════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Looks like you launched a headed browser without having a XServer running.                     ║
║ Set either 'headless: true' or use 'xvfb-run <your-playwright-app>' before running Playwright. ║
║                                                                                                ║
║ <3 Playwright Team                                                                             ║
╚════════════════════════════════════════════════════════════════════════════════════════════════╝
Call log:
  - <launching> /root/.cache/ms-playwright/chromium-1169/chrome-linux/chrome --disable-field-trial-config --disable-background-networking --disable-background-timer-throttling --disable-backgrounding-occluded-windows --disable-back-forward-cache --disable-breakpad --disable-client-side-phishin

In [14]:
import asyncio
from playwright.async_api import async_playwright

async def main():
    refcat = "9923301YJ2792D0001FP"
    base_url = "https://www1.sedecatastro.gob.es/OVCConsultaRC/OVCConsultaRC.aspx"

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36",
            locale="es-ES"
        )
        page = await context.new_page()

        await page.goto(base_url, timeout=60000)

        # Wait a bit longer before looking for fields
        await page.wait_for_selector('input[name="tbRC1"]', timeout=60000)

        await page.fill('input[name="tbRC1"]', refcat[:14])
        await page.fill('input[name="tbRC2"]', refcat[14:])
        await page.click('input[name="btBuscar"]')

        await page.wait_for_selector('text=Datos del Inmueble', timeout=60000)

        photo_link = await page.locator('a:has-text("Foto de Fachada")').get_attribute('href')
        if photo_link:
            photo_url = "https://www1.sedecatastro.gob.es" + photo_link
            print("✅ Image URL found:", photo_url)
        else:
            print("❌ No image available for this parcel.")

        await browser.close()

await main()


TimeoutError: Page.wait_for_selector: Timeout 60000ms exceeded.
Call log:
  - waiting for locator("input[name=\"tbRC1\"]") to be visible
